# Curso básico de Python aplicado à Astronomia
### Laboratório Interinstitucional de e-Astronomia
# Aula XX - Relação Massa-riqueza de Aglomerados de Galáxias
Michel Aguena, LAPP/IN2P3 & LIneA

## Objetivo


Aglomerados de galáxias podem ser ferramentas poderosas para se obter infomação sobre a cosmologia,
mas é necessário fazer a associação entre os aglomerados e os halos de matéria escura. Dentre essas propriedades, está a relação entre a riqueza de aglomerados e a massa dos halos.

## Índice
1. [O que são aglomerados de galáxias](#cluster)
2. [Calibrando uma relação (fazendo um "fit")](#fit)
3. [O espaço de parâmetros](#param)
4. [Calibrando a relação massa-riqueza](#mr)

# 1. O que são aglomerados de galáxias <a class="anchor" id="cluster"></a>

# 2. Calibrando uma relação (fazendo um "fit") <a class="anchor" id="fit"></a>
Como definir qual os valores de uma função se ajustam melhor um conjunto de dados?
Ex:

In [ ]:
import numpy as np
import pylab as plt
from IPython.display import Markdown as md
x = np.array([1, 2, 3, 4, 5])
y = np.array([3, 5, 7, 9, 11])
err = np.random.rand(5)*10

In [ ]:
%matplotlib inline
plt.errorbar(x, y, err, ls='', fmt='.', capsize=3)
plt.grid()
plt.xlabel('x')
plt.ylabel('y')
plt.show()

Qual a melhor função que descreve esses dados?
Uma reta:
\begin{equation}
f(x) = a\; x + b
\end{equation}

In [ ]:
def func(x, a, b):
    return a*x+b

In [ ]:
parametros = [(0, 7), (1, 2), (3, 0),
              (2.5, 1), (2, 6), (-3, 15)]

In [ ]:
%matplotlib inline
plt.errorbar(x, y, err, ls='', fmt='.', capsize=3)
for i, (a, b) in enumerate(parametros):
    plt.plot(x, func(x, a, b),
            zorder=0, label=f'c{i} (a={a}, b={b})')
plt.grid()
plt.legend(ncol=2)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

Como avaliar quantativamente qual o melhor ajuste?

Método do $\chi^2$:

\begin{equation}
\chi^2 = \sum_i \frac{(data_i-modelo_i)^2}{(erro_i)^2}
\end{equation}

In [ ]:
def chi2(data, modelo, erro):
    return sum((data-modelo)**2/erro**2)
# Print computation:
def show_chi2_numbers(x, y, err, parametros):
    out = ''
    for i, (a, b) in enumerate(parametros):
        out += (f'$c{i}:'+\
          ' + '.join([rf'\frac{{({y_}-{func(x_, a, b)})^2}}{{{err_:.2f}^2}}'
                     for x_, y_, err_ in zip(x, y, err)])+\
          f'= {chi2(y, func(x, a, b), err):.2f}$\n\n'
               )
    return md(out)
show_chi2_numbers(x, y, err, parametros)

In [ ]:
%matplotlib inline
plt.errorbar(x, y, err, ls='', fmt='.', capsize=3)
for i, (a, b) in enumerate(parametros):
    plt.plot(x, func(x, a, b),
            zorder=0, label=f'c{i} ({chi2(y, func(x, a, b), err):.2f})')
plt.grid()
plt.legend(ncol=2)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

# 3. O espaço de parâmetros <a class="anchor" id="param"></a>

Avaliar a qualidade do ajuste no espaço de parâmetros

In [ ]:
for i, (a, b) in enumerate(parametros):
    plt.scatter(a, b, color=f'C{i}')
    plt.text(a, b, f'c{i}({chi2(y, func(x, a, b), err):.2f})')
plt.xlabel('a')
plt.ylabel('b')
plt.grid()
plt.show()

E se calculassemos os valores na grade?

In [ ]:
a_vals = np.linspace(-3, 7, 99)
b_vals = np.linspace(-14, 16, 99)
# Calcular valores
a_grid = np.outer(a_vals, b_vals*0+1)
b_grid = np.outer(b_vals, a_vals*0+1).T
chi2_vals = np.array([[chi2(y, func(x, a, b), err) for b in b_vals]
             for a in a_vals])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(a_grid, b_grid, chi2_vals,
                 lw=.5)


for i, (a, b) in enumerate(parametros): 
    ax.scatter(a, b, chi2(y, func(x, a, b), err),
               color=f'C{i}', label=f'c{i}')


ax.set_xlabel('a')
ax.set_ylabel('b')
ax.set_zlabel('$\chi^2$')

ax.legend(ncol=2)

fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.resizable = True

plt.show()

* Likelihood
\begin{equation}
\mathcal{L} \propto \exp{\left(-\frac{\chi^2}{2}\right)}
\end{equation}

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(a_grid, b_grid, np.exp(-chi2_vals/2),
                 lw=.5)

ax.set_xlabel('a')
ax.set_ylabel('b')
ax.set_zlabel('$\mathcal{L}$')

for i, (a, b) in enumerate(parametros): 
    ax.scatter(a, b, np.exp(-chi2(y, func(x, a, b), err)/2),
               color=f'C{i}', label=f'c{i}')


fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.resizable = True

maxlike = np.exp(-chi2_vals/2).max()

plt.show()

# 4. Calibrando a relação massa-riqueza <a class="anchor" id="mr"></a>

A relação entre a massa dos halos e a riqueza dos aglomerados é approximada por uma relação de escala:

\begin{equation}
\left(\frac{M}{M^0}\right) \approx
\left(\frac{N_{gals}}{N_{gals}^0}\right)^\alpha 
\end{equation}

In [ ]:
ngals = np.logspace(1, 3)